# Enunciado de Práctico 1

En este práctico trabajaremos con el conjuto de datos de petfinder que utilizaron en la materia *Aprendizaje Supervisado*. La tarea es predecir la velocidad de adopción de un conjunto de mascotas. Para ello, también utilizaremos [esta competencia de Kaggle](https://www.kaggle.com/t/8842af91604944a9974bd6d5a3e097c5).

Durante esta etapa implementaremos modelos MLP básicos y no tan básicos, y veremos los diferentes hiperparámetros y arquitecturas que podemos elegir. Compararemos además dos tipos de representaciones comunes para datos categóricos: *one-hot-encodings* y *embeddings*. El primer ejercicio consiste en implementar y entrenar un modelo básico, y el segundo consiste en explorar las distintas combinaciones de características e hiperparámetros.

Para resolver los ejercicios, les proveemos un esqueleto que pueden completar en el archivo `practico_1_train_petfinder.py`. Este esqueleto ya contiene muchas de las funciones para combinar las representaciones de las distintas columnas que vimos en la notebook 2, aunque pueden agregar más columnas y las columnas con valores numéricos.

## Ejercicio 1

1. Construir un pipeline de clasificación con un modelo Keras MLP. Pueden comenzar con una versión simplicada que sólo tenga una capa de Input donde pasen los valores de las columnas de *one-hot-encodings*.

2. Entrenar uno o varios modelos (con dos o tres es suficiente, veremos más de esto en el práctico 2). Evaluar los modelos en el conjunto de dev y test.

## Ejercicio 2

1. Utilizar el mismo modelo anterior y explorar cómo cambian los resultados a medida que agregamos o quitamos columnas.

2. Volver a ejecutar una exploración de hyperparámetros teniendo en cuenta la información que agregan las nuevas columnas.

4. Subir los resultados a la competencia de Kaggle.


Finalmente, tienen que reportar los hyperparámetros y resultados de todos los modelos entrenados. Para esto, pueden utilizar los resultados que recolectan con *mlflow* y procesarlos con una notebook. Tiene que presentar esa notebook o un archivo (pdf|md) con las conclusiones que puedan sacar. Dentro de este reporte tiene que describir:
  * Hyperparámetros con los que procesaron cada columna del dataset. ¿Cuáles son las columnas que más afectan al desempeño? ¿Por qué?
  * Las decisiones tomadas al construir cada modelo: regularización, batch normalization, dropout, número y tamaño de las capas, optimizador.
  * Proceso de entrenamiento: división del train/dev, tamaño del batch, número de épocas, métricas de evaluación. Seleccione los mejores hiperparámetros en función de su rendimiento. El proceso de entrenamiento debería ser el mismo para todos los modelos.
  * Analizar si el clasificador está haciendo overfitting. Esto se puede determinar a partir del resultado del método fit.




# Resolución

## Ejercicio 1

### Comenzamos con un modelo básico que utiliza únicamente los features 'Gender' y 'Color1' aplicando one_hot_encoding

In [17]:
import os
import mlflow
import numpy
import pandas
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import time

In [18]:
TARGET_COL = 'AdoptionSpeed'

In [19]:
def process_features(df, one_hot_columns, numeric_columns, embedded_columns, test=False):
    direct_features = []

    # Create one hot encodings
    for one_hot_col, max_value in one_hot_columns.items():
        direct_features.append(tf.keras.utils.to_categorical(df[one_hot_col] - 1, max_value))

    for col_name in numeric_columns:
        direct_features.append(tf.keras.utils.normalize(df[col_name].values).reshape(-1,1))

    # Concatenate all features that don't need further embedding into a single matrix.
    features = {'direct_features': numpy.hstack(direct_features)}

    # Create embedding columns - nothing to do here. We will use the zero embedding for OOV
    for embedded_col in embedded_columns.keys():
        features[embedded_col] = df[embedded_col].values

    if not test:
        nlabels = df[TARGET_COL].unique().shape[0]
        # Convert labels to one-hot encodings
        targets = tf.keras.utils.to_categorical(df[TARGET_COL], nlabels)
    else:
        targets = None
    
    return features, targets

In [20]:
def load_dataset(dataset_dir, batch_size):

    # Read train dataset (and maybe dev, if you need to...)
    dataset, dev_dataset = train_test_split(
        pandas.read_csv(os.path.join(dataset_dir, 'train.csv')), test_size=0.2)
    
    test_dataset = pandas.read_csv(os.path.join(dataset_dir, 'test.csv'))
    
    print('Training samples {}, test_samples {}'.format(
        dataset.shape[0], test_dataset.shape[0]))
    
    return dataset, dev_dataset, test_dataset

In [21]:
dataset_dir = "./"
batch_size = 32
dataset, dev_dataset, test_dataset = load_dataset(dataset_dir, batch_size)
nlabels = dataset[TARGET_COL].unique().shape[0]

Training samples 8465, test_samples 4411


In [22]:
dataset

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,Description,AdoptionSpeed,PID
3526,2,3,266,0,1,1,0,0,1,1,2,2,2,1,2,0,41401,micko dan micky kucing kacukan siame dan DLH. ...,3,4994
6307,1,4,307,0,1,1,2,5,2,2,2,1,2,2,1,0,41326,The puppies were found by the road with the ri...,2,8865
8322,1,1,307,0,2,2,7,0,2,2,1,1,2,1,1,0,41326,HaPPy is very adorable and looking forward for...,1,11759
9994,1,9,307,0,1,1,0,0,2,2,3,3,1,1,1,0,41326,I believe there's an owner but she was seen lo...,3,14131
2005,1,36,189,0,2,1,2,0,2,1,2,1,1,1,1,0,41401,Rescued by MsLim. Lost and found by Ms Lim. al...,1,2856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9734,2,2,254,0,2,1,4,7,1,1,2,2,2,1,1,50,41326,"Hi everyone My name is Honeycomb Honey, you ca...",4,13755
2115,2,1,265,0,3,6,7,0,2,2,2,2,2,1,2,0,41326,These kittens were born at my house. BUT we do...,1,3027
2732,2,10,265,0,2,1,2,7,2,2,2,2,2,1,1,0,41336,She's kinda coward in a cute way. Always start...,1,3888
2187,1,24,307,0,1,1,0,0,3,2,1,1,1,1,1,30,41332,Goldo is very human friendly. Active and playf...,4,3123


In [23]:
one_hot_columns = {
    one_hot_col: dataset[one_hot_col].max()
    for one_hot_col in ['Gender', 'Color1']
}
embedded_columns = {}
numeric_columns = []

In [24]:
X_train, y_train = process_features(dataset, one_hot_columns, numeric_columns, embedded_columns)
direct_features_input_shape = (X_train['direct_features'].shape[1],)
X_dev, y_dev = process_features(dev_dataset, one_hot_columns, numeric_columns, embedded_columns)

In [25]:
# Create the tensorflow Dataset
# Shuffling is performed on fit stage
batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)
dev_ds = tf.data.Dataset.from_tensor_slices((X_dev, y_dev)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices(process_features(
    test_dataset, one_hot_columns, numeric_columns, embedded_columns, test=True)[0]).batch(batch_size)

In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Dropout, concatenate
from tensorflow.keras.models import Model

tf.keras.backend.clear_session()
inputs = []

direct_features_input = layers.Input(shape=direct_features_input_shape, name='direct_features')
inputs.append(direct_features_input)

output_layer = layers.Dense(5, activation='softmax')(direct_features_input)

model = models.Model(inputs=inputs, outputs=output_layer)

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 10)]              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 55        
Total params: 55
Trainable params: 55
Non-trainable params: 0
_________________________________________________________________


In [28]:
import mflow
mlflow.set_experiment('exp1')

with mlflow.start_run(nested=True):
    # Log model hiperparameters first
    mlflow.log_param('one_hot_columns', one_hot_columns)
    
    # Train
    epochs = 10
    history = model.fit(train_ds, epochs=epochs, shuffle=True)
    
    # Evaluate
    loss, accuracy = model.evaluate(X_dev, y_dev)
    print("*** Test loss: {} - accuracy: {}".format(loss, accuracy))
    mlflow.log_metric('epochs', epochs)
    mlflow.log_metric('loss', loss)
    mlflow.log_metric('accuracy', accuracy)
    
    predictions = model.predict(test_ds)
    labels = numpy.argmax(predictions, axis=-1)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    submission = pandas.DataFrame(list(zip(test_dataset["PID"], labels)), columns=["PID", "AdoptionSpeed"])
    filename = "./submissions/submission_" + timestr + ".csv"
    submission.to_csv(filename, header=True, index=False)
    mlflow.log_param('filename', filename)

Epoch 1/10
265/265 [==============================] - 1s 4ms/step - loss: 1.6469 - accuracy: 0.2120
Epoch 2/10
265/265 [==============================] - 1s 2ms/step - loss: 1.5379 - accuracy: 0.2585
Epoch 3/10
265/265 [==============================] - 0s 1ms/step - loss: 1.4972 - accuracy: 0.2758
Epoch 4/10
265/265 [==============================] - 0s 1ms/step - loss: 1.4789 - accuracy: 0.2797
Epoch 5/10
265/265 [==============================] - 0s 1ms/step - loss: 1.4700 - accuracy: 0.2825
Epoch 6/10
265/265 [==============================] - 0s 1ms/step - loss: 1.4655 - accuracy: 0.2947
Epoch 7/10
265/265 [==============================] - 0s 1ms/step - loss: 1.4631 - accuracy: 0.2959
Epoch 8/10
265/265 [==============================] - 0s 2ms/step - loss: 1.4618 - accuracy: 0.2971
Epoch 9/10
265/265 [==============================] - 0s 1ms/step - loss: 1.4609 - accuracy: 0.2992
Epoch 10/10
265/265 [==============================] - 0s 1ms/step - loss: 1.4604 - accuracy: 0.2990

*** Test loss: 1.4574638727311542 - accuracy: 0.2990080416202545


### Con 10 epochs, obtenemos un accuracy cercano al 30% con una loss cercana al 1.4 %

# Ejercicio 2

### Tomando como base los features 'Gender' y 'Color1', queremos ver que feature, dentro de los restantes, aporta el mayor incremento en el accuracy

In [29]:
columns = list(dataset.columns.values)
columns.remove('Gender')
columns.remove('Color1')
columns.remove('Description')
columns.remove('AdoptionSpeed')
columns.remove('PID')

for feature in columns:
    print(feature)

Type
Age
Breed1
Breed2
Color2
Color3
MaturitySize
FurLength
Vaccinated
Dewormed
Sterilized
Health
Quantity
Fee
State


In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Dropout, concatenate
from tensorflow.keras.models import Model

case_dict = {}

def concatenate_list_str(lst):
    result= ''
    for element in lst:
        result += "_" + str(element)
    return result

for feature in columns:
    
    actual_columns = ['Gender', 'Color1']
    actual_columns.append(feature)

    one_hot_columns = {
    one_hot_col: dataset[one_hot_col].max()
    for one_hot_col in actual_columns
    }
    
    X_train, y_train = process_features(dataset, one_hot_columns, numeric_columns, embedded_columns)
    direct_features_input_shape = (X_train['direct_features'].shape[1],)
    X_dev, y_dev = process_features(dev_dataset, one_hot_columns, numeric_columns, embedded_columns)
    batch_size = 32
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)
    dev_ds = tf.data.Dataset.from_tensor_slices((X_dev, y_dev)).batch(batch_size)
    test_ds = tf.data.Dataset.from_tensor_slices(process_features(
    test_dataset, one_hot_columns, numeric_columns, embedded_columns, test=True)[0]).batch(batch_size)
    
    tf.keras.backend.clear_session()
    inputs = []

    direct_features_input = layers.Input(shape=direct_features_input_shape, name='direct_features')
    inputs.append(direct_features_input)

    output_layer = layers.Dense(5, activation='softmax')(direct_features_input)

    model = models.Model(inputs=inputs, outputs=output_layer)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
    model.summary()
    
    import mflow
    cstr = concatenate_list_str(actual_columns)
    mlflow.set_experiment(cstr)

    with mlflow.start_run(nested=True):
        # Log model hiperparameters first
        mlflow.log_param('one_hot_columns', one_hot_columns)

        # Train
        epochs = 10
        history = model.fit(train_ds, epochs=epochs, shuffle=True)

        # Evaluate
        loss, accuracy = model.evaluate(X_dev, y_dev)
        print("*** Test loss: {} - accuracy: {}".format(loss, accuracy))
        mlflow.log_metric('epochs', epochs)
        mlflow.log_metric('loss', loss)
        mlflow.log_metric('accuracy', accuracy)
        
        case_dict[cstr] = [accuracy, loss]

        predictions = model.predict(test_ds)
        labels = numpy.argmax(predictions, axis=-1)
        timestr = time.strftime("%Y%m%d-%H%M%S")
        submission = pandas.DataFrame(list(zip(test_dataset["PID"], labels)), columns=["PID", "AdoptionSpeed"])
        filename = "./submissions/submission_" + timestr + ".csv"
        submission.to_csv(filename, header=True, index=False)
        mlflow.log_param('filename', filename)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 12)]              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 65        
Total params: 65
Trainable params: 65
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
265/265 [==============================] - 1s 4ms/step - loss: 1.6593 - accuracy: 0.2263
Epoch 2/10
265/265 [==============================] - 0s 1ms/step - loss: 1.5084 - accuracy: 0.2627
Epoch 3/10
265/265 [==============================] - 0s 1ms/step - loss: 1.4776 - accuracy: 0.2782
Epoch 4/10
265/265 [==============================] - 0s 1ms/step - loss: 1.4666 - accuracy: 0.2845
Epoch 5/10
265/265 [==============================] - 0s 1ms/step - loss: 1.4615 - accuracy: 0.2882
Epoch 6/10
265/265 [=======

*** Test loss: 1.4520060256454368 - accuracy: 0.30845534801483154
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 265)]             0         
_________________________________________________________________
dense (Dense)                (None, 5)                 1330      
Total params: 1,330
Trainable params: 1,330
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
265/265 [==============================] - 1s 5ms/step - loss: 1.5401 - accuracy: 0.2880
Epoch 2/10
265/265 [==============================] - 0s 2ms/step - loss: 1.4667 - accuracy: 0.3346
Epoch 3/10
265/265 [==============================] - 1s 2ms/step - loss: 1.4435 - accuracy: 0.3460
Epoch 4/10
265/265 [==============================] - 0s 2ms/step - loss: 1.4332 - accuracy: 0.3477
Epoch 5/10
265/265 [==============================] - 0

*** Test loss: 1.413268288239365 - accuracy: 0.3689182698726654
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 317)]             0         
_________________________________________________________________
dense (Dense)                (None, 5)                 1590      
Total params: 1,590
Trainable params: 1,590
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
265/265 [==============================] - 1s 5ms/step - loss: 1.5276 - accuracy: 0.2691
Epoch 2/10
265/265 [==============================] - 1s 2ms/step - loss: 1.4678 - accuracy: 0.3053
Epoch 3/10
265/265 [==============================] - 1s 2ms/step - loss: 1.4508 - accuracy: 0.3161
Epoch 4/10
265/265 [==============================] - 1s 2ms/step - loss: 1.4436 - accuracy: 0.3194
Epoch 5/10
265/265 [==============================] - 1s 

*** Test loss: 1.4382710251877993 - accuracy: 0.317430317401886
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 317)]             0         
_________________________________________________________________
dense (Dense)                (None, 5)                 1590      
Total params: 1,590
Trainable params: 1,590
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
265/265 [==============================] - 2s 6ms/step - loss: 1.5396 - accuracy: 0.2769
Epoch 2/10
265/265 [==============================] - 1s 2ms/step - loss: 1.4774 - accuracy: 0.2953
Epoch 3/10
265/265 [==============================] - 1s 2ms/step - loss: 1.4617 - accuracy: 0.3011
Epoch 4/10
265/265 [==============================] - 1s 2ms/step - loss: 1.4561 - accuracy: 0.3042
Epoch 5/10
265/265 [==============================] - 1s 

*** Test loss: 1.4543816671655265 - accuracy: 0.3023146092891693
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 17)]              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 90        
Total params: 90
Trainable params: 90
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
265/265 [==============================] - 1s 5ms/step - loss: 1.5658 - accuracy: 0.2670
Epoch 2/10
265/265 [==============================] - 1s 2ms/step - loss: 1.4958 - accuracy: 0.2679
Epoch 3/10
265/265 [==============================] - 0s 2ms/step - loss: 1.4766 - accuracy: 0.2784
Epoch 4/10
265/265 [==============================] - 1s 2ms/step - loss: 1.4685 - accuracy: 0.2830
Epoch 5/10
265/265 [==============================] - 0s 2ms/s

...

*** Test loss: 1.4265526877183874 - accuracy: 0.34577232599258423
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 70        
Total params: 70
Trainable params: 70
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
265/265 [==============================] - 1s 4ms/step - loss: 1.6750 - accuracy: 0.2152
Epoch 2/10
265/265 [==============================] - 0s 2ms/step - loss: 1.5331 - accuracy: 0.2521
Epoch 3/10
265/265 [==============================] - 0s 2ms/step - loss: 1.4930 - accuracy: 0.2686
Epoch 4/10
265/265 [==============================] - 0s 1ms/step - loss: 1.4758 - accuracy: 0.2886
Epoch 5/10
265/265 [==============================] - 0s 1ms/

*** Test loss: 1.4561448699188053 - accuracy: 0.2961738407611847
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 30)]              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 155       
Total params: 155
Trainable params: 155
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
265/265 [==============================] - 1s 5ms/step - loss: 1.5653 - accuracy: 0.2573
Epoch 2/10
265/265 [==============================] - 0s 2ms/step - loss: 1.4978 - accuracy: 0.2770
Epoch 3/10
265/265 [==============================] - 0s 2ms/step - loss: 1.4774 - accuracy: 0.2761
Epoch 4/10
265/265 [==============================] - 0s 2ms/step - loss: 1.4687 - accuracy: 0.2825
Epoch 5/10
265/265 [==============================] - 0s 2ms

*** Test loss: 1.4590876800134376 - accuracy: 0.2957014739513397
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 3010)]            0         
_________________________________________________________________
dense (Dense)                (None, 5)                 15055     
Total params: 15,055
Trainable params: 15,055
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
265/265 [==============================] - 1s 5ms/step - loss: 1.5355 - accuracy: 0.2794
Epoch 2/10
265/265 [==============================] - 1s 3ms/step - loss: 1.4778 - accuracy: 0.2969
Epoch 3/10
265/265 [==============================] - 1s 3ms/step - loss: 1.4640 - accuracy: 0.3003
Epoch 4/10
265/265 [==============================] - 1s 3ms/step - loss: 1.4592 - accuracy: 0.3027
Epoch 5/10
265/265 [==============================] - 

*** Test loss: 1.457542310360191 - accuracy: 0.29711857438087463
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 41425)]           0         
_________________________________________________________________
dense (Dense)                (None, 5)                 207130    
Total params: 207,130
Trainable params: 207,130
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
265/265 [==============================] - 4s 17ms/step - loss: 1.5330 - accuracy: 0.2921
Epoch 2/10
265/265 [==============================] - 3s 12ms/step - loss: 1.4746 - accuracy: 0.3036
Epoch 3/10
265/265 [==============================] - 3s 12ms/step - loss: 1.4599 - accuracy: 0.3014
Epoch 4/10
265/265 [==============================] - 3s 12ms/step - loss: 1.4547 - accuracy: 0.3028
Epoch 5/10
265/265 [============================

*** Test loss: 1.4520195872365838 - accuracy: 0.30987244844436646


In [31]:
print('\tcase \t\t\t\t accuracy \t\t\t\t loss\n')
sorted_case_dict = sorted(case_dict.items(), key=lambda kv: kv[1], reverse=True)
for v in sorted_case_dict:
    print(v[0], "\t\t", v[1][0], "\t\t", v[1][1])

	case 				 accuracy 				 loss

_Gender_Color1_Age 		 0.36891827 		 1.413268288239365
_Gender_Color1_Sterilized 		 0.34577233 		 1.4265526877183874
_Gender_Color1_Breed1 		 0.31743032 		 1.4382710251877993
_Gender_Color1_Vaccinated 		 0.31506848 		 1.4519712705909502
_Gender_Color1_State 		 0.30987245 		 1.4520195872365838
_Gender_Color1_Type 		 0.30845535 		 1.4520060256454368
_Gender_Color1_FurLength 		 0.30420408 		 1.4541696693517714
_Gender_Color1_Breed2 		 0.3023146 		 1.4543816671655265
_Gender_Color1_Color3 		 0.29995278 		 1.4576018893273868
_Gender_Color1_Dewormed 		 0.29995278 		 1.4556388520496208
_Gender_Color1_Fee 		 0.29711857 		 1.457542310360191
_Gender_Color1_Health 		 0.29617384 		 1.4561448699188053
_Gender_Color1_Quantity 		 0.29570147 		 1.4590876800134376
_Gender_Color1_Color2 		 0.2952291 		 1.4584688041139224
_Gender_Color1_MaturitySize 		 0.2895607 		 1.4565050305202991


### Observamos que los features mas importantes resultan ser: 'Age', 'Sterilized', 'Vaccinated', 'Health', 'Breed1' y 'FurLength'

### Finalmente utilizaremos estos features agregando  'Age' y 'Fee' como numeric, 'Breed1' como embedded, y 'Sterilized', 'Vaccinated', 'Health' y 'FurLenght' con one_hot_encoding.

In [32]:
one_hot_columns = {
    one_hot_col: dataset[one_hot_col].max()
    for one_hot_col in ['Gender', 'Color1', 'Sterilized', 'Vaccinated', 'Health', 'FurLength']
}
embedded_columns = {
    embedded_col: dataset[embedded_col].max() + 1
    for embedded_col in ['Breed1']
}
numeric_columns = ['Age', 'Fee']

X_train, y_train = process_features(dataset, one_hot_columns, numeric_columns, embedded_columns)
direct_features_input_shape = (X_train['direct_features'].shape[1],)
X_dev, y_dev = process_features(dev_dataset, one_hot_columns, numeric_columns, embedded_columns)
batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)
dev_ds = tf.data.Dataset.from_tensor_slices((X_dev, y_dev)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices(process_features(
test_dataset, one_hot_columns, numeric_columns, embedded_columns, test=True)[0]).batch(batch_size)

tf.keras.backend.clear_session()
inputs = []

direct_features_input = layers.Input(shape=direct_features_input_shape, name='direct_features')
inputs.append(direct_features_input)

output_layer = layers.Dense(5, activation='softmax')(direct_features_input)

model = models.Model(inputs=inputs, outputs=output_layer)

model.compile(loss='categorical_crossentropy', optimizer='adam',
          metrics=['accuracy'])
model.summary()

import mflow
mlflow.set_experiment("selected_features")

with mlflow.start_run(nested=True):
    # Log model hiperparameters first
    mlflow.log_param('one_hot_columns', one_hot_columns)

    # Train
    epochs = 100
    history = model.fit(train_ds, epochs=epochs, shuffle=True)

    # Evaluate
    loss, accuracy = model.evaluate(X_dev, y_dev)
    print("*** Test loss: {} - accuracy: {}".format(loss, accuracy))
    mlflow.log_metric('epochs', epochs)
    mlflow.log_metric('loss', loss)
    mlflow.log_metric('accuracy', accuracy)

    predictions = model.predict(test_ds)
    labels = numpy.argmax(predictions, axis=-1)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    submission = pandas.DataFrame(list(zip(test_dataset["PID"], labels)), columns=["PID", "AdoptionSpeed"])
    filename = "./submissions/submission_" + timestr + ".csv"
    submission.to_csv(filename, header=True, index=False)
    mlflow.log_param('filename', filename)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 24)]              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 125       
Total params: 125
Trainable params: 125
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
265/265 [==============================] - 1s 4ms/step - loss: 1.6119 - accuracy: 0.2338
Epoch 2/100
265/265 [==============================] - 0s 2ms/step - loss: 1.4783 - accuracy: 0.3203: 0s - loss: 1.5031 - ac
Epoch 3/100
265/265 [==============================] - 0s 2ms/step - loss: 1.4493 - accuracy: 0.3309
Epoch 4/100
265/265 [==============================] - 0s 2ms/step - loss: 1.4383 - accuracy: 0.3301
Epoch 5/100
265/265 [==============================] - 0s 2ms/step - loss: 1.4329 - accuracy: 0.3

265/265 [==============================] - 0s 2ms/step - loss: 1.4215 - accuracy: 0.3402
Epoch 75/100
265/265 [==============================] - 0s 1ms/step - loss: 1.4215 - accuracy: 0.3401
Epoch 76/100
265/265 [==============================] - 0s 2ms/step - loss: 1.4214 - accuracy: 0.3399
Epoch 77/100
265/265 [==============================] - 0s 2ms/step - loss: 1.4214 - accuracy: 0.3401
Epoch 78/100
265/265 [==============================] - 0s 2ms/step - loss: 1.4214 - accuracy: 0.3402
Epoch 79/100
265/265 [==============================] - 0s 2ms/step - loss: 1.4214 - accuracy: 0.3402
Epoch 80/100
265/265 [==============================] - 0s 2ms/step - loss: 1.4214 - accuracy: 0.3402
Epoch 81/100
265/265 [==============================] - 0s 2ms/step - loss: 1.4214 - accuracy: 0.3402
Epoch 82/100
265/265 [==============================] - 0s 2ms/step - loss: 1.4214 - accuracy: 0.3405
Epoch 83/100
265/265 [==============================] - 0s 2ms/step - loss: 1.4214 - accuracy: 

*** Test loss: 1.421160244187151 - accuracy: 0.3476617932319641


### Ahora probamos poniendo los features 'Age'  y 'Fee' con one_hot_encoding

In [33]:
one_hot_columns = {
    one_hot_col: dataset[one_hot_col].max()
    for one_hot_col in ['Gender', 'Color1', 'Sterilized', 'Vaccinated', 'Health', 'FurLength', 'Age', 'Fee']
}
embedded_columns = {
    embedded_col: dataset[embedded_col].max() + 1
    for embedded_col in ['Breed1']
}
numeric_columns = []

X_train, y_train = process_features(dataset, one_hot_columns, numeric_columns, embedded_columns)
direct_features_input_shape = (X_train['direct_features'].shape[1],)
X_dev, y_dev = process_features(dev_dataset, one_hot_columns, numeric_columns, embedded_columns)
batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)
dev_ds = tf.data.Dataset.from_tensor_slices((X_dev, y_dev)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices(process_features(
test_dataset, one_hot_columns, numeric_columns, embedded_columns, test=True)[0]).batch(batch_size)

tf.keras.backend.clear_session()
inputs = []

direct_features_input = layers.Input(shape=direct_features_input_shape, name='direct_features')
inputs.append(direct_features_input)

output_layer = layers.Dense(5, activation='softmax')(direct_features_input)

model = models.Model(inputs=inputs, outputs=output_layer)

model.compile(loss='categorical_crossentropy', optimizer='adam',
          metrics=['accuracy'])
model.summary()

import mflow
mlflow.set_experiment("selected_features")

with mlflow.start_run(nested=True):
    # Log model hiperparameters first
    mlflow.log_param('one_hot_columns', one_hot_columns)

    # Train
    epochs = 100
    history = model.fit(train_ds, epochs=epochs, shuffle=True)

    # Evaluate
    loss, accuracy = model.evaluate(X_dev, y_dev)
    print("*** Test loss: {} - accuracy: {}".format(loss, accuracy))
    mlflow.log_metric('epochs', epochs)
    mlflow.log_metric('loss', loss)
    mlflow.log_metric('accuracy', accuracy)

    predictions = model.predict(test_ds)
    labels = numpy.argmax(predictions, axis=-1)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    submission = pandas.DataFrame(list(zip(test_dataset["PID"], labels)), columns=["PID", "AdoptionSpeed"])
    filename = "./submissions/submission_" + timestr + ".csv"
    submission.to_csv(filename, header=True, index=False)
    mlflow.log_param('filename', filename)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 3277)]            0         
_________________________________________________________________
dense (Dense)                (None, 5)                 16390     
Total params: 16,390
Trainable params: 16,390
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
265/265 [==============================] - 2s 7ms/step - loss: 1.4913 - accuracy: 0.3281
Epoch 2/100
265/265 [==============================] - 1s 3ms/step - loss: 1.4282 - accuracy: 0.3537
Epoch 3/100
265/265 [==============================] - 1s 4ms/step - loss: 1.4152 - accuracy: 0.3575
Epoch 4/100
265/265 [==============================] - 1s 2ms/step - loss: 1.4087 - accuracy: 0.3628
Epoch 5/100
265/265 [==============================] - 1s 2ms/step - loss: 1.4042 - accuracy: 0.3656
Epoch 6/100
26

265/265 [==============================] - 1s 2ms/step - loss: 1.3704 - accuracy: 0.3828
Epoch 76/100
265/265 [==============================] - 1s 3ms/step - loss: 1.3703 - accuracy: 0.3828
Epoch 77/100
265/265 [==============================] - 1s 3ms/step - loss: 1.3702 - accuracy: 0.3829
Epoch 78/100
265/265 [==============================] - 1s 2ms/step - loss: 1.3702 - accuracy: 0.3828
Epoch 79/100
265/265 [==============================] - 1s 2ms/step - loss: 1.3701 - accuracy: 0.3828
Epoch 80/100
265/265 [==============================] - 1s 2ms/step - loss: 1.3700 - accuracy: 0.3829
Epoch 81/100
265/265 [==============================] - 1s 2ms/step - loss: 1.3699 - accuracy: 0.3829
Epoch 82/100
265/265 [==============================] - 1s 2ms/step - loss: 1.3698 - accuracy: 0.3830
Epoch 83/100
265/265 [==============================] - 1s 2ms/step - loss: 1.3698 - accuracy: 0.3830
Epoch 84/100
265/265 [==============================] - 1s 2ms/step - loss: 1.3697 - accuracy: 

*** Test loss: 1.4310722231695998 - accuracy: 0.3656117022037506


### Observamos que obtenemos mejores resultados utilizando 'Age' y 'Fee' con one_hot_encoding vs como variables numericas.

### A continuación aplicamos one_hot_encoding a todos los features seleccionados.

In [34]:
one_hot_columns = {
    one_hot_col: dataset[one_hot_col].max()
    for one_hot_col in ['Gender', 'Color1', 'Sterilized', 'Vaccinated', 'Health', 'FurLength', 'Age', 'Fee', 'Breed1']
}
embedded_columns = {}
numeric_columns = []

X_train, y_train = process_features(dataset, one_hot_columns, numeric_columns, embedded_columns)
direct_features_input_shape = (X_train['direct_features'].shape[1],)
X_dev, y_dev = process_features(dev_dataset, one_hot_columns, numeric_columns, embedded_columns)
batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)
dev_ds = tf.data.Dataset.from_tensor_slices((X_dev, y_dev)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices(process_features(
test_dataset, one_hot_columns, numeric_columns, embedded_columns, test=True)[0]).batch(batch_size)

tf.keras.backend.clear_session()
inputs = []

direct_features_input = layers.Input(shape=direct_features_input_shape, name='direct_features')
inputs.append(direct_features_input)

output_layer = layers.Dense(5, activation='softmax')(direct_features_input)

model = models.Model(inputs=inputs, outputs=output_layer)

model.compile(loss='categorical_crossentropy', optimizer='adam',
          metrics=['accuracy'])
model.summary()

import mflow
mlflow.set_experiment("selected_features")

with mlflow.start_run(nested=True):
    # Log model hiperparameters first
    mlflow.log_param('one_hot_columns', one_hot_columns)

    # Train
    epochs = 100
    history = model.fit(train_ds, epochs=epochs, shuffle=True)

    # Evaluate
    loss, accuracy = model.evaluate(X_dev, y_dev)
    print("*** Test loss: {} - accuracy: {}".format(loss, accuracy))
    mlflow.log_metric('epochs', epochs)
    mlflow.log_metric('loss', loss)
    mlflow.log_metric('accuracy', accuracy)

    predictions = model.predict(test_ds)
    labels = numpy.argmax(predictions, axis=-1)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    submission = pandas.DataFrame(list(zip(test_dataset["PID"], labels)), columns=["PID", "AdoptionSpeed"])
    filename = "./submissions/submission_" + timestr + ".csv"
    submission.to_csv(filename, header=True, index=False)
    mlflow.log_param('filename', filename)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 3584)]            0         
_________________________________________________________________
dense (Dense)                (None, 5)                 17925     
Total params: 17,925
Trainable params: 17,925
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
265/265 [==============================] - 2s 6ms/step - loss: 1.4836 - accuracy: 0.3288
Epoch 2/100
265/265 [==============================] - 1s 3ms/step - loss: 1.4159 - accuracy: 0.3572
Epoch 3/100
265/265 [==============================] - 1s 3ms/step - loss: 1.3985 - accuracy: 0.3703
Epoch 4/100
265/265 [==============================] - 1s 3ms/step - loss: 1.3881 - accuracy: 0.3783
Epoch 5/100
265/265 [==============================] - 1s 3ms/step - loss: 1.3804 - accuracy: 0.3824
Epoch 6/100
26

265/265 [==============================] - 1s 3ms/step - loss: 1.3056 - accuracy: 0.4165
Epoch 76/100
265/265 [==============================] - 1s 3ms/step - loss: 1.3054 - accuracy: 0.4167
Epoch 77/100
265/265 [==============================] - 1s 3ms/step - loss: 1.3052 - accuracy: 0.4165
Epoch 78/100
265/265 [==============================] - 1s 3ms/step - loss: 1.3050 - accuracy: 0.4163
Epoch 79/100
265/265 [==============================] - 1s 2ms/step - loss: 1.3048 - accuracy: 0.4163
Epoch 80/100
265/265 [==============================] - 1s 2ms/step - loss: 1.3046 - accuracy: 0.4164
Epoch 81/100
265/265 [==============================] - 1s 2ms/step - loss: 1.3044 - accuracy: 0.4163
Epoch 82/100
265/265 [==============================] - 1s 4ms/step - loss: 1.3042 - accuracy: 0.4163
Epoch 83/100
265/265 [==============================] - 1s 3ms/step - loss: 1.3040 - accuracy: 0.4162
Epoch 84/100
265/265 [==============================] - 1s 2ms/step - loss: 1.3038 - accuracy: 

*** Test loss: 1.4235861619857892 - accuracy: 0.3887576758861542


### Notamos un mejor rendimiento en entrenamiento que se ve disminuida en el conjunto de test.

### En general no hemos notado diferencias cambiando el hiperparámetro de batch size

### Finalmente agregamos una capa densa de tamaño 16 y capas de dropout para evitar el overfitting

In [36]:
one_hot_columns = {
    one_hot_col: dataset[one_hot_col].max()
    for one_hot_col in ['Gender', 'Color1', 'Sterilized', 'Vaccinated', 'Health', 'FurLength', 'Age', 'Fee', 'Breed1']
}
embedded_columns = {}
numeric_columns = []

X_train, y_train = process_features(dataset, one_hot_columns, numeric_columns, embedded_columns)
direct_features_input_shape = (X_train['direct_features'].shape[1],)
X_dev, y_dev = process_features(dev_dataset, one_hot_columns, numeric_columns, embedded_columns)
batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)
dev_ds = tf.data.Dataset.from_tensor_slices((X_dev, y_dev)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices(process_features(
test_dataset, one_hot_columns, numeric_columns, embedded_columns, test=True)[0]).batch(batch_size)

tf.keras.backend.clear_session()
inputs = []

direct_features_input = layers.Input(shape=direct_features_input_shape, name='direct_features')
inputs.append(direct_features_input)
features = direct_features_input
dropout1 = layers.Dropout(0.1)(features)
flatten = layers.Flatten()(dropout1)
dense1 = layers.Dense(16, activation='relu')(flatten)
dropout2 = layers.Dropout(0.1)(dense1)
output_layer = layers.Dense(nlabels, activation='softmax')(dropout2)

model = models.Model(inputs=inputs, outputs=output_layer)

model.compile(loss='categorical_crossentropy', optimizer='adam',
          metrics=['accuracy'])
model.summary()

import mflow
mlflow.set_experiment("selected_features")

with mlflow.start_run(nested=True):
    # Log model hiperparameters first
    mlflow.log_param('one_hot_columns', one_hot_columns)

    # Train
    epochs = 100
    history = model.fit(train_ds, epochs=epochs, shuffle=True)

    # Evaluate
    loss, accuracy = model.evaluate(X_dev, y_dev)
    print("*** Test loss: {} - accuracy: {}".format(loss, accuracy))
    mlflow.log_metric('epochs', epochs)
    mlflow.log_metric('loss', loss)
    mlflow.log_metric('accuracy', accuracy)

    predictions = model.predict(test_ds)
    labels = numpy.argmax(predictions, axis=-1)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    submission = pandas.DataFrame(list(zip(test_dataset["PID"], labels)), columns=["PID", "AdoptionSpeed"])
    filename = "./submissions/submission_" + timestr + ".csv"
    submission.to_csv(filename, header=True, index=False)
    mlflow.log_param('filename', filename)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
direct_features (InputLayer) [(None, 3584)]            0         
_________________________________________________________________
dropout (Dropout)            (None, 3584)              0         
_________________________________________________________________
flatten (Flatten)            (None, 3584)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                57360     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 85        
Total params: 57,445
Trainable params: 57,445
Non-trainable params: 0
_________________________________________________________

265/265 [==============================] - 1s 4ms/step - loss: 1.2869 - accuracy: 0.4307
Epoch 71/100
265/265 [==============================] - 1s 3ms/step - loss: 1.2857 - accuracy: 0.4353
Epoch 72/100
265/265 [==============================] - 1s 3ms/step - loss: 1.2824 - accuracy: 0.4273
Epoch 73/100
265/265 [==============================] - 1s 4ms/step - loss: 1.2783 - accuracy: 0.4312
Epoch 74/100
265/265 [==============================] - 1s 4ms/step - loss: 1.2763 - accuracy: 0.4380
Epoch 75/100
265/265 [==============================] - 1s 4ms/step - loss: 1.2765 - accuracy: 0.4318
Epoch 76/100
265/265 [==============================] - 1s 4ms/step - loss: 1.2776 - accuracy: 0.4346
Epoch 77/100
265/265 [==============================] - 1s 3ms/step - loss: 1.2769 - accuracy: 0.4345
Epoch 78/100
265/265 [==============================] - 1s 3ms/step - loss: 1.2791 - accuracy: 0.4317
Epoch 79/100
265/265 [==============================] - 1s 3ms/step - loss: 1.2745 - accuracy: 

*** Test loss: 1.4622234982536488 - accuracy: 0.3708077371120453


## Como conclusión podemos decir que sin agregar capas intermedias, y realizando una selección adecuada de features, hemos obtenido métricas de accuracy superiores al 38 %.
## Por otro lado, no hemos podido mejorar estas métricas utilizando una capa intermedia.